# SSC5877 - Verificação, Validação e Teste de Software


## Trabalho final - Desafio

### Conceitos

#### O que é um ano bissexto?

Um ano bissexto é aquele que possui 366 dias ao invés de 365, com o dia extra sendo 29 de fevereiro. De acordo com as regras do calendário gregoriano (em vigor desde 1752), um ano é bissexto se:

*   For divisível por 4

*   Exceto se for divisível por 100

*   A menos que também seja divisível por 400

Além disso, anos anteriores a 1752 seguem uma lógica simplificada: são bissextos se forem divisíveis por 4.

#### O que é MC/DC?

MC/DC (Modified Condition/Decision Coverage) é um critério de teste que garante que:

*   Cada condição atômica em uma decisão (ex: a and b) foi avaliada como True e False;

*   Cada condição afeta isoladamente o resultado final da decisão.


### Funções

#### Monitoramento

In [ ]:
log_execucao = []

def monitorar(condicao_str, contexto):
    try:
        resultado = eval(condicao_str, {}, contexto)
        log_execucao.append((condicao_str, resultado))
        return resultado
    except Exception as e:
        log_execucao.append((condicao_str, f"Erro: {e}"))
        return False


#### Bissexto

In [ ]:
codigo_eh_bissexto = '''
def eh_bissexto(ano: int) -> bool:
    if ano < 1 or ano > 9999:
        raise ValueError("Ano inválido: deve estar entre 1 e 9999.")

    if ano <= 1752:
        return ano % 4 == 0

    if ano % 400 == 0:
        return True
    if ano % 100 == 0:
        return False
    return ano % 4 == 0
'''


In [ ]:
def eh_bissexto(ano: int) -> bool:
    contexto = {"ano": ano}

    if monitorar("ano < 1 or ano > 9999", contexto):
        raise ValueError("Ano inválido: deve estar entre 1 e 9999.")

    if monitorar("ano <= 1752", contexto):
        return monitorar("ano % 4 == 0", contexto)

    if monitorar("ano % 400 == 0", contexto):
        return True
    if monitorar("ano % 100 == 0", contexto):
        return False
    return monitorar("ano % 4 == 0", contexto)


#### Extração de decisões

In [ ]:
import ast

def extrair_decisoes(codigo: str):
    arvore = ast.parse(codigo)
    decisoes = []

    class VisitanteDecisao(ast.NodeVisitor):
        def visit_If(self, node):
            decisoes.append(ast.unparse(node.test))
            self.generic_visit(node)

    VisitanteDecisao().visit(arvore)
    return decisoes

decisoes = extrair_decisoes(codigo_eh_bissexto)
for i, d in enumerate(decisoes, 1):
    print(f"Decisão {i}: {d}")


Decisão 1: ano < 1 or ano > 9999
Decisão 2: ano <= 1752
Decisão 3: ano % 400 == 0
Decisão 4: ano % 100 == 0


#### Geração de combinações MC/DC

In [ ]:
import itertools

def gerar_mcdc(decisao):
    operadores_logicos = ["and", "or"]
    for operador in operadores_logicos:
        if operador in decisao:
            termos = [t.strip(" ()") for t in decisao.split(operador)]
            combinacoes = list(itertools.product([True, False], repeat=len(termos)))
            validas = []

            for i, c1 in enumerate(combinacoes):
                for j, c2 in enumerate(combinacoes):
                    if i >= j:
                        continue
                    difs = [k for k in range(len(c1)) if c1[k] != c2[k]]
                    if len(difs) == 1:
                        r1 = eval(f" {operador} ".join(str(v) for v in c1))
                        r2 = eval(f" {operador} ".join(str(v) for v in c2))
                        if r1 != r2:
                            validas.append((c1, r1))
                            validas.append((c2, r2))

            unicos = []
            for entrada in validas:
                if entrada not in unicos:
                    unicos.append(entrada)
            return termos, unicos

    return [decisao], [((True,), True), ((False,), False)]

for decisao in decisoes:
    termos, entradas = gerar_mcdc(decisao)
    print(f"Decisão: {decisao}")
    print("Combinações a validar (entrada -> saída):")
    for entrada, resultado in entradas:
        print(f"  {entrada} -> {resultado}")
    print()


Decisão: ano < 1 or ano > 9999
Combinações a validar (entrada -> saída):
  (True, False) -> True
  (False, False) -> False
  (False, True) -> True

Decisão: ano <= 1752
Combinações a validar (entrada -> saída):
  (True,) -> True
  (False,) -> False

Decisão: ano % 400 == 0
Combinações a validar (entrada -> saída):
  (True,) -> True
  (False,) -> False

Decisão: ano % 100 == 0
Combinações a validar (entrada -> saída):
  (True,) -> True
  (False,) -> False



#### Validação das combinações


In [ ]:
def avaliar_entrada(ano, termos, operador):
    valores = [eval(t, {}, {"ano": ano}) for t in termos]
    if operador:
        resultado = eval(f" {operador} ".join(str(v) for v in valores))
    else:
        resultado = valores[0]
    return tuple(valores), resultado

def extrair_termos_operador(decisao):
    if " and " in decisao:
        return [t.strip() for t in decisao.split("and")], "and"
    elif " or " in decisao:
        return [t.strip() for t in decisao.split("or")], "or"
    else:
        return [decisao.strip()], None

def verificar_cobertura_mcdc_real(decisoes, anos):
    print("Verificação da cobertura MC/DC com base nos anos testados:\n")

    for decisao in decisoes:
        termos, entradas_esperadas = gerar_mcdc(decisao)
        termos_completos, operador = extrair_termos_operador(decisao)

        entradas_reais = set()
        for ano in anos:
            try:
                valores, resultado = avaliar_entrada(ano, termos_completos, operador)
                entradas_reais.add((valores, resultado))
            except Exception:
                continue

        total = len(entradas_esperadas)
        cobertas = 0
        faltando = []

        for entrada, saida in entradas_esperadas:
            if (entrada, saida) in entradas_reais:
                cobertas += 1
            else:
                faltando.append((entrada, saida))

        print(f"Decisão: {decisao}")
        print("Combinações necessárias:")
        for e, r in entradas_esperadas:
            print(f"  {e} -> {r}")
        print(f"Cobertura atingida: {cobertas}/{total}")

        if cobertas == total:
            print("Cobertura MC/DC completa\n")
        else:
            print("Cobertura MC/DC incompleta")
            print("Faltando as combinações:")
            for e, r in faltando:
                print(f"  {e} -> {r}")
            print()


#### Função de execução


In [ ]:
log_execucao = []

def executar_mcdc(anos):
    cobertura_decisao = {}

    for ano in anos:
        log_execucao.clear()
        try:
            resultado = eh_bissexto(ano)
            print(f"Ano {ano}: {'Bissexto' if resultado else 'Não bissexto'}")
        except ValueError as e:
            print(f"Ano {ano}: Erro - {e}")

        print("Condições avaliadas:")
        for cond, val in log_execucao:
            print(f"  {cond} => {val}")
            cobertura_decisao.setdefault(cond, set()).add(val)

        print("-" * 40)

    return cobertura_decisao


### Testes

#### Testes estáticos

In [ ]:
anos = [-1, 1600, 1700, 1752, 1800, 1900, 2000, 2020, 2023, 10000]

cobertura = executar_mcdc(anos)

print("Anos testados:")
print(anos)

print("-" * 40)

print("\nResumo da Cobertura:")

for cond, valores in cobertura.items():
    print(f"'{cond}': coberto {valores}")


Ano -1: Erro - Ano inválido: deve estar entre 1 e 9999.
Condições avaliadas:
  ano < 1 or ano > 9999 => True
----------------------------------------
Ano 1600: Bissexto
Condições avaliadas:
  ano < 1 or ano > 9999 => False
  ano <= 1752 => True
  ano % 4 == 0 => True
----------------------------------------
Ano 1700: Bissexto
Condições avaliadas:
  ano < 1 or ano > 9999 => False
  ano <= 1752 => True
  ano % 4 == 0 => True
----------------------------------------
Ano 1752: Bissexto
Condições avaliadas:
  ano < 1 or ano > 9999 => False
  ano <= 1752 => True
  ano % 4 == 0 => True
----------------------------------------
Ano 1800: Não bissexto
Condições avaliadas:
  ano < 1 or ano > 9999 => False
  ano <= 1752 => False
  ano % 400 == 0 => False
  ano % 100 == 0 => True
----------------------------------------
Ano 1900: Não bissexto
Condições avaliadas:
  ano < 1 or ano > 9999 => False
  ano <= 1752 => False
  ano % 400 == 0 => False
  ano % 100 == 0 => True
------------------------------

In [ ]:
verificar_cobertura_mcdc_real(decisoes, anos)


Verificação da cobertura MC/DC com base nos anos testados:

Decisão: ano < 1 or ano > 9999
Combinações necessárias:
  (True, False) -> True
  (False, False) -> False
  (False, True) -> True
Cobertura atingida: 3/3
Cobertura MC/DC completa

Decisão: ano <= 1752
Combinações necessárias:
  (True,) -> True
  (False,) -> False
Cobertura atingida: 2/2
Cobertura MC/DC completa

Decisão: ano % 400 == 0
Combinações necessárias:
  (True,) -> True
  (False,) -> False
Cobertura atingida: 2/2
Cobertura MC/DC completa

Decisão: ano % 100 == 0
Combinações necessárias:
  (True,) -> True
  (False,) -> False
Cobertura atingida: 2/2
Cobertura MC/DC completa



#### Testes aleatórios

In [ ]:
import random

log_execucao = []

anos = [random.randint(1, 9999) for _ in range(100)]

cobertura = executar_mcdc(anos)

print("Anos testados:")
print(anos)

print("-" * 40)

print("\nResumo:")

for cond, valores in cobertura.items():
    print(f"'{cond}': coberto {valores}")


Ano 7845: Não bissexto
Condições avaliadas:
  ano < 1 or ano > 9999 => False
  ano <= 1752 => False
  ano % 400 == 0 => False
  ano % 100 == 0 => False
  ano % 4 == 0 => False
----------------------------------------
Ano 620: Bissexto
Condições avaliadas:
  ano < 1 or ano > 9999 => False
  ano <= 1752 => True
  ano % 4 == 0 => True
----------------------------------------
Ano 5470: Não bissexto
Condições avaliadas:
  ano < 1 or ano > 9999 => False
  ano <= 1752 => False
  ano % 400 == 0 => False
  ano % 100 == 0 => False
  ano % 4 == 0 => False
----------------------------------------
Ano 4559: Não bissexto
Condições avaliadas:
  ano < 1 or ano > 9999 => False
  ano <= 1752 => False
  ano % 400 == 0 => False
  ano % 100 == 0 => False
  ano % 4 == 0 => False
----------------------------------------
Ano 9341: Não bissexto
Condições avaliadas:
  ano < 1 or ano > 9999 => False
  ano <= 1752 => False
  ano % 400 == 0 => False
  ano % 100 == 0 => False
  ano % 4 == 0 => False
---------------

In [ ]:
verificar_cobertura_mcdc_real(decisoes, anos)


Verificação da cobertura MC/DC com base nos anos testados:

Decisão: ano < 1 or ano > 9999
Combinações necessárias:
  (True, False) -> True
  (False, False) -> False
  (False, True) -> True
Cobertura atingida: 1/3
Cobertura MC/DC incompleta
Faltando as combinações:
  (True, False) -> True
  (False, True) -> True

Decisão: ano <= 1752
Combinações necessárias:
  (True,) -> True
  (False,) -> False
Cobertura atingida: 2/2
Cobertura MC/DC completa

Decisão: ano % 400 == 0
Combinações necessárias:
  (True,) -> True
  (False,) -> False
Cobertura atingida: 1/2
Cobertura MC/DC incompleta
Faltando as combinações:
  (True,) -> True

Decisão: ano % 100 == 0
Combinações necessárias:
  (True,) -> True
  (False,) -> False
Cobertura atingida: 2/2
Cobertura MC/DC completa



### Conclusões

***Adicionar as conclusões e as observações das implementações feitas***

# Implementação Alternativa

Autor: Caio

Não tenho certeza se entendi o que era pra fazer, mas seguem os códigos do ano bissexto e teste MC/DC.

Bissexto

In [ ]:
def eh_bissexto(ano: int) -> bool:
    if ano < 1 or ano > 9999:
        raise ValueError("O ano deve estar entre 1 e 9999.")
    if ano <= 1752:
        return ano % 4 == 0
    if ano % 400 == 0:
        return True
    if ano % 100 == 0:
        return False
    return ano % 4 == 0

Teste

In [ ]:
import unittest
from main import eh_bissexto


class TestEhBissextoMCDC(unittest.TestCase):

    # --- Testes para a Decisão 1: if ano < 1 or ano > 9999 ---

    def test_decisao1_condicao_A_mostra_independencia(self):
        # Teste 1: ano = 0 (A=True, B=False) -> Gera exceção
        with self.assertRaises(ValueError, msg="Deveria lançar exceção para ano < 1"):
            eh_bissexto(0)

        # Teste 2: ano = 2000 (A=False, B=False) -> Não gera exceção
        try:
            eh_bissexto(2000)
        except ValueError:
            self.fail("Não deveria lançar exceção para ano=2000")

    def test_decisao1_condicao_B_mostra_independencia(self):
        # Teste 1: ano = 10000 (A=False, B=True) -> Gera exceção
        with self.assertRaises(ValueError, msg="Deveria lançar exceção para ano > 9999"):
            eh_bissexto(10000)

        # Teste 2: ano = 2000 (A=False, B=False) -> Não gera exceção
        try:
            eh_bissexto(2000)
        except ValueError:
            self.fail("Não deveria lançar exceção para ano=2000")

    # --- Testes para os caminhos de execução após a validação ---

    def test_caminho_juliano_nao_bissexto(self):
        # ano=1751: D2 (ano <= 1752) é True, D2a (ano % 4 == 0) é False.
        self.assertFalse(eh_bissexto(1751), "Falhou para ano juliano não bissexto (1751)")

    def test_caminho_juliano_bissexto(self):
        # ano=1752: D2 (ano <= 1752) é True, D2a (ano % 4 == 0) é True.
        self.assertTrue(eh_bissexto(1752), "Falhou para ano juliano bissexto (1752)")

    def test_caminho_gregoriano_div_por_100_nao_400(self):
        # ano=1900: D2(F), D3(F), D4(T) -> Retorna False.
        self.assertFalse(eh_bissexto(1900), "Falhou para ano secular não bissexto (1900)")

    def test_caminho_gregoriano_div_por_400(self):
        # ano=2000: D2(F), D3(T) -> Retorna True.
        self.assertTrue(eh_bissexto(2000), "Falhou para ano secular bissexto (2000)")

    def test_caminho_gregoriano_div_por_4(self):
        # ano=2024: D4(F), D5(T) -> Retorna True.
        self.assertTrue(eh_bissexto(2024), "Falhou para ano comum bissexto (2024)")

    def test_caminho_gregoriano_nao_bissexto(self):
        # ano=2025: D5(F) -> Retorna False.
        self.assertFalse(eh_bissexto(2025), "Falhou para ano não bissexto (2025)")


if __name__ == '__main__':
    unittest.main(verbosity=2)

Comando pra rodar a budega

In [ ]:
python -m unittest -v test_mcdc_bissexto.py